 Imports and setup

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam

Load dataset

In [ ]:
df = pd.read_csv('/content/electricitydemanddata.csv')
print(df.columns.tolist())
print(df.head())

['Date', 'Day of the week', 'Year', 'Month', 'Max. Demand at eve. peak (Generation end)', 'Max. Demand at eve. peak (Sub-station end)', 'Highest Generation (Generation end)', 'Minimum Generation (Generation end)', 'Day-peak Generation (Generation end)', 'Evening-peak Generation (Generation end)', 'Minimum Generation Forecast up to 8:00 hrs.', 'Maximum Temperature in Dhaka was', 'Gas/LF limitation', 'Coal supply Limitation', 'Low water level in Kaptai lake', 'Plants under shut down/ maintenance', 'Dhaka_demand', 'Dhaka_supply', 'Dhaka_load', 'Chattogram_demand', 'Chattogram_supply', 'Chattogram_load', 'Rajshahi_demand', 'Rajshahi_supply', 'Rajshahi_load', 'Mymensingh_demand', 'Mymensingh_supply', 'Mymensingh_load', 'Sylhet_demand', 'Sylhet_supply', 'Sylhet_load', 'Barishal_demand', 'Barishal_supply', 'Barishal_load', 'Rangpur_demand', 'Rangpur_supply', 'Rangpur_load', 'Cumilla_demand', 'Cumilla_supply', 'Cumilla_load', 'Khulna_demand', 'Khulna_supply', 'Khulna_load', 'Holiday name', 'Ho

 Preprocessing

In [ ]:
# Convert boolean columns to float
for col in df.columns:
    if df[col].dtype == 'bool':
        df[col] = df[col].astype(float)

# Drop non-numeric columns
df = df.drop(columns=['Date', 'Holiday name'])

# One-hot encode categorical columns
df = pd.get_dummies(df, columns=['Day of the week', 'Month', 'Holiday_cat'], drop_first=True)

# Identify feature and target columns
target_cols = [col for col in df.columns if '_demand' in col]
feature_cols = [col for col in df.columns if col not in target_cols]

# Features and target arrays
X = df[feature_cols].values.astype(np.float32)
y = df[target_cols].values.astype(np.float32)

# Scale separately
feature_scaler = MinMaxScaler()
target_scaler = MinMaxScaler()

X = feature_scaler.fit_transform(X)
y = target_scaler.fit_transform(y)



# 4️⃣ Create multi-step GRU dataset


In [ ]:
def create_gru_dataset_multioutput(X, y, lookback=21, horizon=2):
    Xs, ys = [], []
    for i in range(lookback, len(X)-horizon+1):
        Xs.append(X[i-lookback:i])
        ys.append(y[i:i+horizon])
    return np.array(Xs, dtype=np.float32), np.array(ys, dtype=np.float32)

lookback = 21  # input window
horizon = 2    # 48-hour forecast (2 days)

X_gru, y_gru = create_gru_dataset_multioutput(X, y, lookback, horizon)
print("X shape:", X_gru.shape)
print("y shape:", y_gru.shape)


X shape: (1828, 21, 51)
y shape: (1828, 2, 9)


# 5️⃣ Train / Validation / Test split

In [ ]:
total_samples = len(X_gru)
train_end = int(total_samples * 0.7)
val_end = int(total_samples * 0.85)

X_train, y_train = X_gru[:train_end], y_gru[:train_end]
X_val, y_val = X_gru[train_end:val_end], y_gru[train_end:val_end]
X_test, y_test = X_gru[val_end:], y_gru[val_end:]

print("Train:", X_train.shape, "Validation:", X_val.shape, "Test:", X_test.shape)

num_features = X_train.shape[2]
num_targets = y_train.shape[2]

Train: (1279, 21, 51) Validation: (274, 21, 51) Test: (275, 21, 51)


# 6️⃣ Build GRU model

In [ ]:
model = Sequential()
model.add(GRU(64, input_shape=(lookback, num_features), return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(32, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(horizon * num_targets))  # flatten output

model.compile(optimizer='adam', loss='mse')
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru_2 (GRU)                     │ (None, 21, 64)         │        22,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 21, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_3 (GRU)                     │ (None, 32)             │         9,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 18)             │           594 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32,466 (126.82 KB)

 Trainable params: 32,466 (126.82 KB)

 Non-trainable params: 0 (0.00 B)

# 7️⃣ Train GRU model

In [ ]:
history = model.fit(
    X_train, y_train.reshape(y_train.shape[0], horizon*num_targets),
    epochs=50,
    batch_size=32,
    verbose=2
)

Epoch 1/50
40/40 - 5s - 127ms/step - loss: 0.0675
Epoch 2/50
40/40 - 1s - 25ms/step - loss: 0.0285
Epoch 3/50
40/40 - 1s - 31ms/step - loss: 0.0225
Epoch 4/50
40/40 - 1s - 25ms/step - loss: 0.0194
Epoch 5/50
40/40 - 1s - 25ms/step - loss: 0.0173
Epoch 6/50
40/40 - 2s - 39ms/step - loss: 0.0154
Epoch 7/50
40/40 - 2s - 40ms/step - loss: 0.0145
Epoch 8/50
40/40 - 2s - 49ms/step - loss: 0.0144
Epoch 9/50
40/40 - 1s - 25ms/step - loss: 0.0131
Epoch 10/50
40/40 - 1s - 25ms/step - loss: 0.0123
Epoch 11/50
40/40 - 1s - 25ms/step - loss: 0.0114
Epoch 12/50
40/40 - 1s - 25ms/step - loss: 0.0113
Epoch 13/50
40/40 - 1s - 31ms/step - loss: 0.0108
Epoch 14/50
40/40 - 1s - 25ms/step - loss: 0.0103
Epoch 15/50
40/40 - 1s - 25ms/step - loss: 0.0099
Epoch 16/50
40/40 - 1s - 30ms/step - loss: 0.0096
Epoch 17/50
40/40 - 2s - 40ms/step - loss: 0.0094
Epoch 18/50
40/40 - 1s - 32ms/step - loss: 0.0093
Epoch 19/50
40/40 - 1s - 24ms/step - loss: 0.0086
Epoch 20/50
40/40 - 1s - 33ms/step - loss: 0.0086
Epoch 21

# 8️⃣ Evaluate on Train Set

In [ ]:
y_train_pred = model.predict(X_train).reshape(y_train.shape[0], horizon, num_targets)
y_train_inv = target_scaler.inverse_transform(y_train.reshape(-1, num_targets)).reshape(y_train.shape)
y_train_pred_inv = target_scaler.inverse_transform(y_train_pred.reshape(-1, num_targets)).reshape(y_train_pred.shape)

mae_train = mean_absolute_error(y_train_inv.reshape(-1, num_targets), y_train_pred_inv.reshape(-1, num_targets))
rmse_train = np.sqrt(mean_squared_error(y_train_inv.reshape(-1, num_targets), y_train_pred_inv.reshape(-1, num_targets)))
mape_train = np.mean(np.abs((y_train_inv - y_train_pred_inv)/y_train_inv)) * 100
r2_train = r2_score(y_train_inv.reshape(-1, num_targets), y_train_pred_inv.reshape(-1, num_targets))

print("Train Metrics:")
print(f"MAE: {mae_train:.2f}, RMSE: {rmse_train:.2f}, MAPE: {mape_train:.2f}%, R²: {r2_train:.2f}")


40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step
Train Metrics:
MAE: 60.71, RMSE: 101.74, MAPE: 5.33%, R²: 0.86


# 9️⃣ Evaluate on Validation Set

In [ ]:
y_val_pred = model.predict(X_val).reshape(y_val.shape[0], horizon, num_targets)
y_val_inv = target_scaler.inverse_transform(y_val.reshape(-1, num_targets)).reshape(y_val.shape)
y_val_pred_inv = target_scaler.inverse_transform(y_val_pred.reshape(-1, num_targets)).reshape(y_val_pred.shape)

mae_val = mean_absolute_error(y_val_inv.reshape(-1, num_targets), y_val_pred_inv.reshape(-1, num_targets))
rmse_val = np.sqrt(mean_squared_error(y_val_inv.reshape(-1, num_targets), y_val_pred_inv.reshape(-1, num_targets)))
mape_val = np.mean(np.abs((y_val_inv - y_val_pred_inv)/y_val_inv)) * 100
r2_val = r2_score(y_val_inv.reshape(-1, num_targets), y_val_pred_inv.reshape(-1, num_targets))

print("Validation Metrics:")
print(f"MAE: {mae_val:.2f}, RMSE: {rmse_val:.2f}, MAPE: {mape_val:.2f}%, R²: {r2_val:.2f}")



9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Validation Metrics:
MAE: 86.73, RMSE: 140.12, MAPE: 7.32%, R²: 0.71


# 10️⃣ Evaluate on Test Set

In [ ]:
y_test_pred = model.predict(X_test).reshape(y_test.shape[0], horizon, num_targets)
y_test_inv = target_scaler.inverse_transform(y_test.reshape(-1, num_targets)).reshape(y_test.shape)
y_test_pred_inv = target_scaler.inverse_transform(y_test_pred.reshape(-1, num_targets)).reshape(y_test_pred.shape)

mae_test = mean_absolute_error(y_test_inv.reshape(-1, num_targets), y_test_pred_inv.reshape(-1, num_targets))
rmse_test = np.sqrt(mean_squared_error(y_test_inv.reshape(-1, num_targets), y_test_pred_inv.reshape(-1, num_targets)))
mape_test = np.mean(np.abs((y_test_inv - y_test_pred_inv)/y_test_inv)) * 100
r2_test = r2_score(y_test_inv.reshape(-1, num_targets), y_test_pred_inv.reshape(-1, num_targets))

print("Test Metrics:")
print(f"MAE: {mae_test:.2f}, RMSE: {rmse_test:.2f}, MAPE: {mape_test:.2f}%, R²: {r2_test:.2f}")


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Test Metrics:
MAE: 106.32, RMSE: 168.14, MAPE: 8.35%, R²: 0.58


Random Search using Keras Tuner

Import Keras Tuner

In [ ]:
!pip install keras-tuner --quiet

import keras_tuner as kt
from tensorflow.keras.optimizers import Adam


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 3.3 MB/s eta 0:00:00


Build GRU model for tuning

In [ ]:
def build_gru_model(hp):
    num_features = X_train.shape[2]
    num_targets = y_train.shape[2]

    model = Sequential()

    # GRU Layer 1
    model.add(GRU(
        units=hp.Int('units1', min_value=32, max_value=128, step=32),
        input_shape=(lookback, num_features),
        return_sequences=True
    ))
    model.add(Dropout(hp.Float('dropout1', min_value=0.1, max_value=0.5, step=0.1)))

    # GRU Layer 2
    model.add(GRU(
        units=hp.Int('units2', min_value=16, max_value=64, step=16),
        return_sequences=False
    ))
    model.add(Dropout(hp.Float('dropout2', min_value=0.1, max_value=0.5, step=0.1)))

    # Dense output layer (flatten for multi-step)
    model.add(Dense(horizon * num_targets))

    # Compile
    model.compile(
        optimizer=Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mse'
    )

    return model


Initialize Random Search tuner

In [ ]:
tuner = kt.RandomSearch(
    build_gru_model,
    objective='val_loss',
    max_trials=10,  # number of hyperparameter combinations to try
    executions_per_trial=1,
    overwrite=True,
    directory='gru_tuner',
    project_name='regional_demand_gru'
)

tuner.search_space_summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Search space summary
Default search space size: 5
units1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
dropout1 (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
units2 (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': 'linear'}
dropout2 (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


Run the tuner

In [ ]:
tuner.search(
    X_train, y_train.reshape(y_train.shape[0], horizon*num_targets),
    epochs=20,
    batch_size=32,
    validation_data=(X_val, y_val.reshape(y_val.shape[0], horizon*num_targets)),
    verbose=2
)


Trial 10 Complete [00h 00m 30s]
val_loss: 0.01756625436246395

Best val_loss So Far: 0.006831677630543709
Total elapsed time: 00h 06m 48s


Get best hyperparameters and build model

In [ ]:
best_hp = tuner.get_best_hyperparameters()[0]
print("Best Hyperparameters Found:")
print(best_hp.values)

# Build the best model using best hyperparameters
best_gru_model = tuner.hypermodel.build(best_hp)


Best Hyperparameters Found:
{'units1': 128, 'dropout1': 0.1, 'units2': 64, 'dropout2': 0.4, 'learning_rate': 0.001}


Train best GRU model on training set

In [ ]:
history = best_gru_model.fit(
    X_train,
    y_train.reshape(y_train.shape[0], horizon*num_targets),  # flatten multi-step target
    epochs=50,
    batch_size=32,
    verbose=2
)


Epoch 1/50
40/40 - 5s - 135ms/step - loss: 0.0666
Epoch 2/50
40/40 - 2s - 39ms/step - loss: 0.0292
Epoch 3/50
40/40 - 3s - 77ms/step - loss: 0.0226
Epoch 4/50
40/40 - 2s - 58ms/step - loss: 0.0191
Epoch 5/50
40/40 - 2s - 38ms/step - loss: 0.0170
Epoch 6/50
40/40 - 2s - 38ms/step - loss: 0.0159
Epoch 7/50
40/40 - 2s - 38ms/step - loss: 0.0145
Epoch 8/50
40/40 - 2s - 38ms/step - loss: 0.0133
Epoch 9/50
40/40 - 2s - 38ms/step - loss: 0.0127
Epoch 10/50
40/40 - 2s - 41ms/step - loss: 0.0115
Epoch 11/50
40/40 - 3s - 67ms/step - loss: 0.0112
Epoch 12/50
40/40 - 1s - 37ms/step - loss: 0.0110
Epoch 13/50
40/40 - 2s - 38ms/step - loss: 0.0103
Epoch 14/50
40/40 - 2s - 38ms/step - loss: 0.0101
Epoch 15/50
40/40 - 1s - 37ms/step - loss: 0.0098
Epoch 16/50
40/40 - 1s - 37ms/step - loss: 0.0090
Epoch 17/50
40/40 - 1s - 37ms/step - loss: 0.0090
Epoch 18/50
40/40 - 4s - 94ms/step - loss: 0.0090
Epoch 19/50
40/40 - 2s - 39ms/step - loss: 0.0088
Epoch 20/50
40/40 - 2s - 38ms/step - loss: 0.0085
Epoch 21

Evaluate on Train

In [ ]:
y_train_pred = best_gru_model.predict(X_train).reshape(y_train.shape[0], horizon, num_targets)
y_train_inv = target_scaler.inverse_transform(y_train.reshape(-1, num_targets)).reshape(y_train.shape)
y_train_pred_inv = target_scaler.inverse_transform(y_train_pred.reshape(-1, num_targets)).reshape(y_train_pred.shape)

mae_train = mean_absolute_error(y_train_inv.reshape(-1, num_targets), y_train_pred_inv.reshape(-1, num_targets))
rmse_train = np.sqrt(mean_squared_error(y_train_inv.reshape(-1, num_targets), y_train_pred_inv.reshape(-1, num_targets)))
mape_train = np.mean(np.abs((y_train_inv - y_train_pred_inv)/y_train_inv)) * 100
r2_train = r2_score(y_train_inv.reshape(-1, num_targets), y_train_pred_inv.reshape(-1, num_targets))

print("Train Metrics:")
print(f"MAE: {mae_train:.2f}, RMSE: {rmse_train:.2f}, MAPE: {mape_train:.2f}%, R²: {r2_train:.2f}")


40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step
Train Metrics:
MAE: 57.57, RMSE: 97.59, MAPE: 5.08%, R²: 0.87


Evaluate on Validation

In [ ]:
y_val_pred = best_gru_model.predict(X_val).reshape(y_val.shape[0], horizon, num_targets)
y_val_inv = target_scaler.inverse_transform(y_val.reshape(-1, num_targets)).reshape(y_val.shape)
y_val_pred_inv = target_scaler.inverse_transform(y_val_pred.reshape(-1, num_targets)).reshape(y_val_pred.shape)

mae_val = mean_absolute_error(y_val_inv.reshape(-1, num_targets), y_val_pred_inv.reshape(-1, num_targets))
rmse_val = np.sqrt(mean_squared_error(y_val_inv.reshape(-1, num_targets), y_val_pred_inv.reshape(-1, num_targets)))
mape_val = np.mean(np.abs((y_val_inv - y_val_pred_inv)/y_val_inv)) * 100
r2_val = r2_score(y_val_inv.reshape(-1, num_targets), y_val_pred_inv.reshape(-1, num_targets))

print("Validation Metrics after Random Search:")
print(f"MAE: {mae_val:.2f}, RMSE: {rmse_val:.2f}, MAPE: {mape_val:.2f}%, R²: {r2_val:.2f}")


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Validation Metrics after Random Search:
MAE: 80.55, RMSE: 131.74, MAPE: 6.66%, R²: 0.76


Define Hyperparameter Grid

In [ ]:
# Hyperparameter grid for GRU
units1_list = [32, 128]
units2_list = [16, 64]
dropout1_list = [0.1, 0.3]
dropout2_list = [0.1, 0.3]
learning_rate_list = [1e-3]


Grid Search Loop for GRU

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error

best_val_mae = float('inf')
best_params = None
best_model = None

for units1 in units1_list:
    for units2 in units2_list:
        for dropout1 in dropout1_list:
            for dropout2 in dropout2_list:
                for lr in learning_rate_list:

                    # Build GRU model
                    model = Sequential()
                    model.add(GRU(
                        units1,
                        input_shape=(lookback, X_train.shape[2]),
                        return_sequences=True
                    ))
                    model.add(Dropout(dropout1))

                    model.add(GRU(
                        units2,
                        return_sequences=False
                    ))
                    model.add(Dropout(dropout2))

                    model.add(Dense(horizon * num_targets))

                    model.compile(
                        optimizer=Adam(learning_rate=lr),
                        loss='mse'
                    )

                    # Train on TRAIN set only
                    model.fit(
                        X_train,
                        y_train.reshape(y_train.shape[0], horizon * num_targets),
                        epochs=20,
                        batch_size=32,
                        verbose=0
                    )

                    # ----- Validation -----
                    y_val_pred = model.predict(X_val)
                    y_val_pred = y_val_pred.reshape(y_val.shape[0], horizon, num_targets)

                    y_val_inv = target_scaler.inverse_transform(
                        y_val.reshape(-1, num_targets)
                    ).reshape(y_val.shape)

                    y_val_pred_inv = target_scaler.inverse_transform(
                        y_val_pred.reshape(-1, num_targets)
                    ).reshape(y_val_pred.shape)

                    val_mae = mean_absolute_error(
                        y_val_inv.reshape(-1, num_targets),
                        y_val_pred_inv.reshape(-1, num_targets)
                    )

                    print(
                        f"units1:{units1}, units2:{units2}, "
                        f"d1:{dropout1}, d2:{dropout2}, "
                        f"lr:{lr}, val_MAE:{val_mae:.2f}"
                    )

                    # Save best model
                    if val_mae < best_val_mae:
                        best_val_mae = val_mae
                        best_params = (units1, units2, dropout1, dropout2, lr)
                        best_model = model


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step
units1:32, units2:16, d1:0.1, d2:0.1, lr:0.001, val_MAE:88.08


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step
units1:32, units2:16, d1:0.1, d2:0.3, lr:0.001, val_MAE:91.27


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 77ms/step
units1:128, units2:64, d1:0.1, d2:0.3, lr:0.001, val_MAE:77.16


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step
units1:128, units2:64, d1:0.3, d2:0.1, lr:0.001, val_MAE:94.15


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 76ms/step
units1:128, units2:64, d1:0.3, d2:0.3, lr:0.001, val_MAE:86.28


Print Best Hyperparameters

In [ ]:
print("Best Hyperparameters Found (GRU Grid Search):")
print(f"units1: {best_params[0]}")
print(f"units2: {best_params[1]}")
print(f"dropout1: {best_params[2]}")
print(f"dropout2: {best_params[3]}")
print(f"learning_rate: {best_params[4]}")
print(f"Best Validation MAE: {best_val_mae:.2f}")


Best Hyperparameters Found (GRU Grid Search):
units1: 128
units2: 64
dropout1: 0.1
dropout2: 0.3
learning_rate: 0.001
Best Validation MAE: 77.16
